# Cooperative race

Before you start working with this notebook, remember to:

* Copy racers into `Cooperative Designs`
* Remove all directories `coop_*` if any exist

You also need to create a file called `students.json`, with an entry for every student who might submit, in the following format:

```json
[
    {
        "first": "Somefirstname",
        "last": "Somelastname",
        "netid": "somenetid"
    },
    {
        "first": "Anotherfirstname",
        "last": "Anotherlastname",
        "netid": "anothernetid"
    }
]
```

You can think of the syntax here as a list of dict's. Note that (unlike in python), there must *not* be a comma after the last element of the list or after the last element of each dict.

Import modules and configure the notebook.

In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import secrets
import json
import shutil
import subprocess
import ae353_drone
import importlib
importlib.reload(ae353_drone)

Create and print seed so it is possible to reproduce the results.

In [ ]:
seed = secrets.randbits(32)
print(seed)

Create simulator.

In [ ]:
simulator = ae353_drone.Simulator(display=True, seed=seed)

Load student roster.

In [ ]:
with open('students.json', 'r') as infile:
    students = json.load(infile)

def get_student(students, netid):
    for student in students:
        if student['netid'] == netid:
            return student
    return None

Define source directory with all designs.

In [ ]:
srcdir_designs = 'Cooperative Designs'

Make sure all files in source directory have lower-case names.

In [ ]:
srcdir = srcdir_designs
for file in os.listdir(srcdir):
    os.rename(os.path.join(srcdir, file), os.path.join(srcdir, file.lower()))

Make sure all PNG files in source directory really are PNG files.

In [ ]:
srcdir = srcdir_designs
template_image = 'question_mark.png'
for file in os.listdir(srcdir):
    if file.endswith('.png'):
        completed_process = subprocess.run([
                    'convert',
                    os.path.join(srcdir, file),
                    os.path.join(srcdir, file),
                ], capture_output=True)
        if completed_process.returncode == 0:
            print(f' converted {file}')
        else:
            print(f' FAILED to convert {file} (returncode: {completed_process.returncode}), replacing with template')
            shutil.copyfile(template_image, os.path.join(srcdir, file))

Load drones from source directory, overriding the maximum allowable number.

In [ ]:
simulator.clear_drones()
failures = simulator.load_drones(srcdir_designs, no_max_num_drones=True)

List disqualified drones.

In [ ]:
print(f'DISQUALIFIED ({len(failures)}):\n')
for failure in failures:
    student = get_student(students, failure)
    if student is None:
        name = ''
    else:
        name = f'{student["first"]} {student["last"]}'
    print(f' {failure:10s} : {name}')

List qualified drones.

In [ ]:
print(f'QUALIFIED ({len(simulator.drones)}):\n')
for drone in simulator.drones:
    student = get_student(students, drone['name'])
    if student is None:
        raise Exception(f'could not find student for this drone name: {drone["name"]}')
    print(f' {drone["name"]:10s} : {student["first"]} {student["last"]}')
    
qualified = [drone['name'] for drone in simulator.drones]

Create empty list of non-finishers.

In [ ]:
did_not_finish = []

Define index of first race.

In [ ]:
index_of_race = 0

Choose number of drones to add each time.

In [ ]:
num_drones_to_add = 10

## Set up cooperative race

Create directory in which to put racers.

In [ ]:
index_of_race += 1
print(f'Setting up cooperative race {index_of_race} with {len(qualified)} candidate racers')

racedir = f'coop_{index_of_race}'
os.mkdir(racedir)

# Get list of qualified racers
all_racers = qualified.copy()

# Shuffle order of this list
simulator.rng.shuffle(all_racers)

## Run cooperative race

Each time you run this code, more racers will be added. If everyone finishes the race, keep adding until you get them all! If some racers do not finish, they will be removed from consideration, and you should go back to "set up cooperative race" again before proceeding.

Add more racers to race.

In [ ]:
racers = all_racers[-num_drones_to_add:]
all_racers = all_racers[:-num_drones_to_add]
for racer in racers:
    shutil.copyfile(os.path.join(srcdir_designs, f'{racer}.py'), os.path.join(racedir, f'{racer}.py'))
    shutil.copyfile(os.path.join(srcdir_designs, f'{racer}.png'), os.path.join(racedir, f'{racer}.png'))

Ready...

In [ ]:
# Clear drones
simulator.clear_drones()

# Move rings
simulator.move_rings()

# Load drones
simulator.load_drones(racedir)

# Reset
simulator.reset()

Steady...

In [ ]:
simulator.camera_contestview()

num_drones = len(simulator.drones)
num_columns = 3
num_rows = np.ceil(num_drones / num_columns).astype(int)
fig, axs = plt.subplots(num_rows, num_columns, figsize=(12, 4 * num_rows))
[ax.set_axis_off() for ax in axs.flatten()]
for ax, drone in zip(axs.flatten(), simulator.drones):
    student = get_student(students, drone['name'])
    if student is None:
        raise Exception(f'could not find student for this drone name: {drone["name"]}')
    im = plt.imread(os.path.join(srcdir, f'{drone["name"]}.png'))
    ax.imshow(im, aspect='equal')
    ax.set_title(f'{drone["name"]}\n{student["first"]} {student["last"]}', fontsize=24)
    ax.axis('equal')

fig.tight_layout(h_pad=5)

Go!

In [ ]:
start_time = time.time()
simulator.run(max_time=45.0, contestview=True)
print(f'real time elapsed: {time.time() - start_time}')

Did everyone finish?

In [ ]:
print('Who finished?\n')
all_finished = True
for drone in simulator.drones:
    if drone['finish_time'] is None:
        all_finished = False
        print(f' {drone["name"]:10s} did not finish ({"too slow, or crashed" if drone["running"] else "error"})')
        qualified.remove(drone['name'])
        did_not_finish.append(drone['name'])
    else:
        print(f' {drone["name"]:10s} finished')
print('\n')
        
if all_finished:
    print(f'Everyone finished!\n')
    if len(all_racers) == 0:
        print(f'No more racers to add! Everybody wins!!')
    else:
        print(f'Add more racers and try again!')
else:
    print(f'Not everyone finished. Do a fresh setup and try again!')